In [300]:
import pandas as pd

In [301]:
from multiprocessing import Pool, cpu_count, current_process

In [302]:
from joblib import Parallel, delayed

In [303]:
import json

In [304]:
import sys

In [305]:
import os

In [306]:
os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [307]:
import tensorflow as tf

In [308]:
import numpy as np

In [309]:
import os

In [310]:
from tqdm import tqdm

In [311]:
import cv2

In [312]:
temp = []

for class_name in ['r', 'p', 'b', 'bl', 'g']:
    df = pd.read_csv('/home/vladimir/workspace/data/kaggle_seals/results/comp4_det_test_patches_s0_{class_name}.txt'.format(class_name=class_name) ,
                   header=None, 
                   sep=' ')
    df.columns = ['file_index', 'confidence', 'x_min', 'y_min', 'x_max', 'y_max']
    
    df['class_name'] = class_name
    
    temp += [df]
    
    

In [313]:
for class_name in ['r', 'p', 'b', 'bl', 'g']:
    df = pd.read_csv('/home/vladimir/workspace/data/kaggle_seals/results/comp4_det_test_patches_s1_{class_name}.txt'.format(class_name=class_name) ,
                   header=None, 
                   sep=' ')
    df.columns = ['file_index', 'confidence', 'x_min', 'y_min', 'x_max', 'y_max']
    
    df['class_name'] = class_name
    
    temp += [df]

In [314]:
df = pd.concat(temp)

In [315]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name
0,2483_0_728.jpg,0.949,635.5,1.0,666.1,59.3,r
1,13682_1808_872.jpg,0.016,811.8,419.0,866.2,487.8,r
2,13682_1808_872.jpg,0.003,892.1,12.1,1000.0,83.7,r
3,6951_975_0.jpg,0.992,666.0,122.4,716.5,153.2,r
4,6951_975_0.jpg,0.001,656.8,143.3,706.6,170.5,r


In [316]:
df.shape

(2302176, 7)

In [317]:
df = df[df['confidence'] > 0.6]

In [318]:
df.shape

(771316, 7)

In [319]:
# df['file_name'] = df['file_index'].copy()

In [320]:
df['file_index'] = df['file_index'].str.replace('.jpg', '')

In [321]:
df['x_shift'] = df['file_index'].str.split('_').str.get(1).astype(int)
df['y_shift'] = df['file_index'].str.split('_').str.get(2).astype(int)

In [322]:
df['x_min'] += df['x_shift']
df['x_max'] += df['x_shift']

df['y_min'] += df['y_shift']
df['y_max'] += df['y_shift']


In [323]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,x_shift,y_shift
0,2483_0_728,0.949,635.5,729.0,666.1,787.3,r,0,728
3,6951_975_0,0.992,1641.0,122.4,1691.5,153.2,r,975,0
6,3500_975_920,0.986,1105.5,1460.2,1158.7,1570.6,r,975,920
18,16679_0_872,0.833,762.4,1317.6,794.0,1374.5,r,0,872
21,13136_975_0,0.998,1793.8,630.6,1862.7,701.9,r,975,0


In [324]:
df['test_id'] = df['file_index'].str.split('_').str.get(0)

In [325]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,x_shift,y_shift,test_id
0,2483_0_728,0.949,635.5,729.0,666.1,787.3,r,0,728,2483
3,6951_975_0,0.992,1641.0,122.4,1691.5,153.2,r,975,0,6951
6,3500_975_920,0.986,1105.5,1460.2,1158.7,1570.6,r,975,920,3500
18,16679_0_872,0.833,762.4,1317.6,794.0,1374.5,r,0,872,16679
21,13136_975_0,0.998,1793.8,630.6,1862.7,701.9,r,975,0,13136


In [326]:
df.shape

(771316, 10)

In [327]:
# sample = pd.DataFrame(pd.read_csv('../data/sample_submission.csv')['test_id'])
# sample['x'] = 1

In [328]:
# sample.info()

In [329]:
df['test_id'] = df['test_id'].astype(int)

In [330]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,x_shift,y_shift,test_id
0,2483_0_728,0.949,635.5,729.0,666.1,787.3,r,0,728,2483
3,6951_975_0,0.992,1641.0,122.4,1691.5,153.2,r,975,0,6951
6,3500_975_920,0.986,1105.5,1460.2,1158.7,1570.6,r,975,920,3500
18,16679_0_872,0.833,762.4,1317.6,794.0,1374.5,r,0,872,16679
21,13136_975_0,0.998,1793.8,630.6,1862.7,701.9,r,975,0,13136


In [331]:
class_name_dict = {'r': 'adult_males',
                  'b': 'adult_females',
                   'p': 'subadult_males',
                  'bl': 'juveniles',
                  'g': 'pups'}

In [332]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,x_shift,y_shift,test_id
0,2483_0_728,0.949,635.5,729.0,666.1,787.3,r,0,728,2483
3,6951_975_0,0.992,1641.0,122.4,1691.5,153.2,r,975,0,6951
6,3500_975_920,0.986,1105.5,1460.2,1158.7,1570.6,r,975,920,3500
18,16679_0_872,0.833,762.4,1317.6,794.0,1374.5,r,0,872,16679
21,13136_975_0,0.998,1793.8,630.6,1862.7,701.9,r,975,0,13136


In [333]:
df['id'] = df['test_id'].astype(str) + '_' + df['class_name']

In [334]:
df.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,x_shift,y_shift,test_id,id
0,2483_0_728,0.949,635.5,729.0,666.1,787.3,r,0,728,2483,2483_r
3,6951_975_0,0.992,1641.0,122.4,1691.5,153.2,r,975,0,6951,6951_r
6,3500_975_920,0.986,1105.5,1460.2,1158.7,1570.6,r,975,920,3500,3500_r
18,16679_0_872,0.833,762.4,1317.6,794.0,1374.5,r,0,872,16679,16679_r
21,13136_975_0,0.998,1793.8,630.6,1862.7,701.9,r,975,0,13136,13136_r


In [335]:
g = df.groupby('id')

In [336]:
def nms(name_groups):
    config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
    dfs = []
    
    with tf.Session(config=config) as sess:
        for idx, name_group in enumerate(name_groups):
#             if idx % 100 == 0:
#                 print("in process {} progress {} of {}".format(current_process(), idx, len(name_groups)))
            name, group = name_group
            boxes_ph = tf.placeholder(np.float32, (None, 4), name='boxes')
            scores_ph = tf.placeholder(np.float32, (None, ), name='confs')
            
            boxes = group[['x_min', 'y_min', 'x_max', 'y_max']]
            scores = group['confidence']
            res = sess.run(tf.image.non_max_suppression(boxes_ph, scores_ph, 1000, 0.2),
                           feed_dict={boxes_ph: boxes, scores_ph: scores})
            r = group.values[res]
            dfs.append(r)
    return dfs

In [337]:
# a0 = nms(chunks[0])

In [338]:
# nms(chunks[0])

In [339]:
num_cpu = cpu_count()

In [340]:
grouped = df.groupby('id')
grouped = [(name, group) for name, group in grouped]
l = len(grouped)
sp = l // num_cpu
chunks = [grouped[i:i + sp] for i in range(0, l, sp)]


In [341]:
result = Parallel(n_jobs=8)(delayed(nms)(r) for r in chunks)

In [342]:
len(result)

8

In [343]:
pd.DataFrame(result[0][0])

,0,1,2,3,4,5,6,7,8,9,10
0,0_0_0,0.999,934.4,850.2,988.1,877.8,b,0,0,0,0_b
1,0_0_0,0.998,904.4,844.4,926.2,895.7,b,0,0,0,0_b
2,0_0_0,0.996,952.3,807.8,990.9,841.2,b,0,0,0,0_b
3,0_0_0,0.995,874.6,818.5,922.4,846.3,b,0,0,0,0_b
4,0_0_0,0.995,770.4,881.9,811.8,918.1,b,0,0,0,0_b
5,0_0_0,0.988,836.6,829.1,857.9,882.5,b,0,0,0,0_b
6,0_0_920,0.987,792,932.8,838.4,963.4,b,0,920,0,0_b
7,0_0_0,0.98,795.8,904.4,839.3,931,b,0,0,0,0_b
8,0_0_0,0.969,921.9,885.4,950.2,922.7,b,0,0,0,0_b
9,0_0_0,0.949,798.1,806,826.8,855.4,b,0,0,0,0_b


In [344]:
tmp = []
for c in tqdm(result):
    for r in c:       
        
        tmp += [pd.DataFrame(r)]

100%|██████████| 8/8 [00:04<00:00,  1.66it/s]


In [345]:
df1 = pd.concat(tmp).reset_index(drop=True)

In [346]:
df1.columns = df.columns

In [347]:
df1.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,x_shift,y_shift,test_id,id
0,0_0_0,0.999,934.4,850.2,988.1,877.8,b,0,0,0,0_b
1,0_0_0,0.998,904.4,844.4,926.2,895.7,b,0,0,0,0_b
2,0_0_0,0.996,952.3,807.8,990.9,841.2,b,0,0,0,0_b
3,0_0_0,0.995,874.6,818.5,922.4,846.3,b,0,0,0,0_b
4,0_0_0,0.995,770.4,881.9,811.8,918.1,b,0,0,0,0_b


In [348]:
df1['file_name'] = df1['test_id'].astype(str) + '.jpg'

In [349]:
df1.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,x_shift,y_shift,test_id,id,file_name
0,0_0_0,0.999,934.4,850.2,988.1,877.8,b,0,0,0,0_b,0.jpg
1,0_0_0,0.998,904.4,844.4,926.2,895.7,b,0,0,0,0_b,0.jpg
2,0_0_0,0.996,952.3,807.8,990.9,841.2,b,0,0,0,0_b,0.jpg
3,0_0_0,0.995,874.6,818.5,922.4,846.3,b,0,0,0,0_b,0.jpg
4,0_0_0,0.995,770.4,881.9,811.8,918.1,b,0,0,0,0_b,0.jpg


In [350]:
colors = {'r': (0, 0, 255),
         'g': (0, 255, 0),
         'bl': (255, 0, 0),
         'p': (127, 0, 255),
#          'l_blue': (255, 255, 0),
#          'd_purple': (255, 0, 127),
         'b': (0, 255, 255),
#          'orange': (0, 128, 255),
#          'l_green': (51, 102, 0)
         }

In [351]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [352]:
# for file_name in tqdm(df1['file_name'].unique()[:100]):
for file_name in ['3386.jpg', '1585.jpg']:
    img = cv2.imread(os.path.join('/home/vladimir/workspace/data/kaggle_seals/Test_small', file_name))
    df_c = df1[df1['file_name'] == file_name]
    for i in df_c.index:
        class_name = df_c.loc[i, 'class_name']
        probs = df_c.loc[i, 'confidence']        
        boxes = df_c.loc[i, ['x_min', 'y_min', 'x_max', 'y_max']].astype(int).values        
#         for j, value in enumerate(boxes):            
        (x_min, y_min, x_max, y_max) = boxes
        img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), colors[class_name], 2)
        img = cv2.putText(img, str(int(100 * probs)), (x_max, y_min), font, 1, colors[class_name], 2)
    cv2.imwrite(os.path.join('test_preds', file_name), img)

In [353]:
sample = pd.read_csv('../data/sample_submission.csv')

In [354]:
sample.head()

,test_id,adult_males,subadult_males,adult_females,juveniles,pups
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0


In [355]:
df1.head()

,file_index,confidence,x_min,y_min,x_max,y_max,class_name,x_shift,y_shift,test_id,id,file_name
0,0_0_0,0.999,934.4,850.2,988.1,877.8,b,0,0,0,0_b,0.jpg
1,0_0_0,0.998,904.4,844.4,926.2,895.7,b,0,0,0,0_b,0.jpg
2,0_0_0,0.996,952.3,807.8,990.9,841.2,b,0,0,0,0_b,0.jpg
3,0_0_0,0.995,874.6,818.5,922.4,846.3,b,0,0,0,0_b,0.jpg
4,0_0_0,0.995,770.4,881.9,811.8,918.1,b,0,0,0,0_b,0.jpg


In [356]:
g = pd.DataFrame(df1.groupby(['test_id', 'class_name'])['x_min'].count()).reset_index()

In [357]:
g['class_name'] = g['class_name'].map(class_name_dict)

In [358]:
df2 = g.pivot(index='test_id', columns='class_name', values='x_min')

In [359]:
df2 = df2.reset_index()

In [360]:
df2.head()

class_name,test_id,adult_females,adult_males,juveniles,pups,subadult_males
0,0,14.0,1.0,16.0,NaN,NaN
1,1,2.0,NaN,2.0,NaN,4.0
2,2,1.0,NaN,NaN,2.0,1.0
3,3,NaN,NaN,1.0,NaN,1.0
4,4,3.0,NaN,NaN,2.0,4.0


In [361]:
sample = pd.DataFrame(pd.read_csv('../data/sample_submission.csv')['test_id'])
sample['x'] = 1

In [362]:
sample = sample.merge(df2, on='test_id', how='left', left_index=True)

In [363]:
sample = sample.drop('x', 1).fillna(0)

In [364]:
for column in sample.columns:
    sample[column] = sample[column].astype(int)

In [365]:
sample.to_csv('../data/c_06.csv', index=False)